In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18  
from pytorchtools import EarlyStopping

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# データの前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10の場合、標準化を行うことが一般的です
])

# CIFAR-10のデータセットをロード
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_minisize = 2000  # 例として100サンプルに制限します
train_dataset = torch.utils.data.Subset(train_dataset, list(range(train_minisize)))
train_size = int(0.8 * len(train_dataset))
val_size= len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)

# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)



Files already downloaded and verified


In [32]:

# ResNetモデルの構築（事前学習済みの重みを使用）
model = resnet18(pretrained=True)
early_stopping = EarlyStopping(patience=3)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 損失関数と最適化手法の定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# デバイスの設定（GPUが利用可能ならGPUを使う）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# トレーニングと評価
num_epochs = 100

for epoch in range(num_epochs):
    # トレーニング
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}")

    # 評価セットで評価
    model.eval()
    val_loss=0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss+=loss.item()
    early_stopping(val_loss, model)
    print(f"Epoch {epoch+1}/{num_epochs}, val Loss: {val_loss/len(val_loader)}")
    if early_stopping.early_stop:
        break # 打ち切り
    



# # テストセットで最終評価
# model.eval()
# correct_test = 0
# total_test = 0

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         total_test += labels.size(0)
#         correct_test += (predicted == labels).sum().item()

# accuracy_test = correct_test / total_test
# print(f"Accuracy on test set: {100 * accuracy_test:.2f}%")

Epoch 1/100, Training Loss: 1.955132007598877
Epoch 1/100, val Loss: 3.1604976058006287
Epoch 2/100, Training Loss: 1.0314557827436006
Epoch 2/100, val Loss: 1.5919855237007141
Epoch 3/100, Training Loss: 0.5017078977364761
Epoch 3/100, val Loss: 1.3436456322669983
Epoch 4/100, Training Loss: 0.2392246316258724
EarlyStopping counter: 1 out of 3
Epoch 4/100, val Loss: 1.5472572147846222
Epoch 5/100, Training Loss: 0.16246346842784148
Epoch 5/100, val Loss: 1.3342233896255493
Epoch 6/100, Training Loss: 0.2058644793354548
EarlyStopping counter: 1 out of 3
Epoch 6/100, val Loss: 1.9289982914924622
Epoch 7/100, Training Loss: 0.22886816412210464


KeyboardInterrupt: 

In [9]:
# ResNetモデルの構築（事前学習済みの重みを使用）
model = resnet18(pretrained=True)
early_stopping = EarlyStopping(patience=3)
# 最終の全結合層をCIFAR-10のクラス数に置き換える
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 損失関数と最適化手法の定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # lrは学習率です

# デバイスの設定（GPUが利用可能ならGPUを使う）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# トレーニング
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# テスト
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {100 * accuracy:.2f}%")

c:\Users\osako\PycharmProjects\pythonProject2\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\osako\PycharmProjects\pythonProject2\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\osako/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 25.3MB/s]


Epoch 1/10, Loss: 2.3164950609207153
Epoch 2/10, Loss: 1.2031625509262085
Epoch 3/10, Loss: 0.5130728483200073
Epoch 4/10, Loss: 0.157041534781456
Epoch 5/10, Loss: 0.069574736058712
Epoch 6/10, Loss: 0.1178677398711443
Epoch 7/10, Loss: 0.08413501549512148
Epoch 8/10, Loss: 0.021268969867378473
Epoch 9/10, Loss: 0.13997943745926023
Epoch 10/10, Loss: 0.030954875517636538
Accuracy on test set: 28.23%
